In [64]:
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [65]:
# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# datasets
trainset = torchvision.datasets.FashionMNIST('../../data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('../../data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')


In [66]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [67]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.90)
scheduler = optim.lr_scheduler.StepLR(optimizer, 1000,gamma=0.9)

In [68]:
running_loss = 0.0
for epoch in range(3):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:    # every 1000 mini-batches...
            scheduler.step()
            print("[%d,%6d] ava_loss: %.5f"%(epoch+1,i+1,running_loss/1000))
            running_loss = 0.0
print('Finished Training')

[1,  1000] ava_loss: 1.76959
[1,  2000] ava_loss: 0.78833
[1,  3000] ava_loss: 0.71147
[1,  4000] ava_loss: 0.65232
[1,  5000] ava_loss: 0.59532
[1,  6000] ava_loss: 0.56110
[1,  7000] ava_loss: 0.52627
[1,  8000] ava_loss: 0.52778
[1,  9000] ava_loss: 0.50799
[1, 10000] ava_loss: 0.48979
[1, 11000] ava_loss: 0.44199
[1, 12000] ava_loss: 0.44952
[1, 13000] ava_loss: 0.44371
[1, 14000] ava_loss: 0.40988
[1, 15000] ava_loss: 0.41333
[2,  1000] ava_loss: 0.40849
[2,  2000] ava_loss: 0.40296
[2,  3000] ava_loss: 0.38900
[2,  4000] ava_loss: 0.37342
[2,  5000] ava_loss: 0.38451
[2,  6000] ava_loss: 0.36791
[2,  7000] ava_loss: 0.35554
[2,  8000] ava_loss: 0.35122
[2,  9000] ava_loss: 0.36175
[2, 10000] ava_loss: 0.35990
[2, 11000] ava_loss: 0.36147
[2, 12000] ava_loss: 0.35014
[2, 13000] ava_loss: 0.34131
[2, 14000] ava_loss: 0.36538
[2, 15000] ava_loss: 0.31961
[3,  1000] ava_loss: 0.34262
[3,  2000] ava_loss: 0.31947
[3,  3000] ava_loss: 0.31791
[3,  4000] ava_loss: 0.33345
[3,  5000] ava

In [69]:
PATH="../../pth/fasion_mnist_test.pth"
torch.save(net.state_dict(),PATH)

In [70]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [71]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network: %d %%' % (
    100 * correct / total))

Accuracy of the network: 87 %


In [72]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of T-shirt/top : 76 %
Accuracy of Trouser : 96 %
Accuracy of Pullover : 79 %
Accuracy of Dress : 88 %
Accuracy of  Coat : 87 %
Accuracy of Sandal : 91 %
Accuracy of Shirt : 68 %
Accuracy of Sneaker : 95 %
Accuracy of   Bag : 96 %
Accuracy of Ankle Boot : 96 %
